#### Models have been generated and saved; time to run them against Test  
Test consists of the features (testMaster) and the labels (rulMaster)

In [ ]:
import pandas as pd
import numpy  as np
import random
import glob
import pickle
import matplotlib.pyplot as plt
from setPandasOptions import setPandasOptions
from getConfig import getConfig
from calcMAPE import calcMAPE

#setPandasOptions()

In [ ]:
config = getConfig()

df = pd.read_csv(config["dataLoc"] + config["testFile"])

In [ ]:
df.set_index("unit", inplace=True)
grp = df.groupby(level=0)

In [ ]:
def getMAPE(actuals, preds):
    return abs((actuals[:10]-preds[:10])/actuals[:10]).mean()

In [ ]:
def processModel(model):
    '''
    df is several operational cycles for a unit
    Some time after the last cycle, failure occurred
    The idea is to predict the RUL, after the last cycle
    '''
    errors = {}
    comparisons = {}
    for unit, cycles in grp:
        actuals = cycles["RUL"]
        del cycles["RUL"]
        preds = model.predict(cycles)
        error = calcMAPE(actuals, preds)
        errors[unit] = error
        comparisons[unit] = (actuals, preds)
    return errors, comparisons

In [ ]:
def getResults(errors):
    best = 0.1
    mid = 0.2
    worst = .35
    tier1 = {k: v for k, v in errors.items() if v < best}
    tier2 = {k: v for k, v in errors.items() if v >= best and v < mid}
    tier3 = {k: v for k, v in errors.items() if v >= mid and v < worst}
    tier4 = {k: v for k, v in errors.items() if v > worst}
    print("{:<8}{:<10}{}".format("Tier", "Cutoff", "Count"))
    print("{:<8}{:<10.0%}{}".format(1,best,len(tier1)))
    print("{:<8}{:<10.0%}{}".format(2,mid,len(tier2)))
    print("{:<8}{:<10.0%}{}".format(3,worst,len(tier3)))
    print("{:<8}{}{:<9.0%}{}".format(4,">",worst,len(tier4)))

In [ ]:
def getModels():
    return glob.glob("/home/tbrownex/RFmodel*")

In [ ]:
models = getModels()
for fname in models:
    print("model: ", fname)
    model = pickle.load(open(fname, 'rb'))
    errors, comparisons = processModel(model)
    getResults(errors)

In [ ]:
def plotComps(comparisons, x=None):
    if x is None:
        x = random.randint(1,len(comparisons))
    actual = np.array(comparisons[x][0])
    prediction = comparisons[x][1]
    plt.plot(actual, label="actual")
    plt.plot(prediction, label="prediction")
    plt.legend(loc='upper right')
    unit = "Unit # " + str(x)
    plt.annotate(unit, xy=(0.05, 0.05), xycoords='axes fraction')
    plt.show()

In [ ]:
'''while True:
    plotComps(comparisons)
    input()'''

In [ ]:
tier4 = {k: v for k, v in errors.items() if v > .35}

In [ ]:
for unit in tier4.keys():
    plotComps(comparisons, unit)
    input()